In [ ]:
%pip install -U --quiet langchain-google-genai langchainhub langgraph google-ai-generativelanguage==0.6.15

In [ ]:
import os
import getpass
from langgraph.graph import StateGraph,END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

In [ ]:
os.environ['GOOGLE_API_KEY']=getpass.getpass("Enter the gemini Gemini API key: ")

Enter the gemini Gemini API key: ··········


In [ ]:
llm=ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest",temperature=0.2)

In [ ]:
def get_input(state:dict)->dict:
  val=input("Enter input:")
  state["val"]=val
  return state

In [ ]:
def input_classifier(state:dict)->dict:
  prompt=(
      f"Classify the given input {state['val']} into one of the categories: \n1.Product (e.g., plastic bottle)\n2.Activity (e.g., long showers)\n3.Transport (e.g., daily car commute)\n4.Energy use (e.g., incandescent bulbs)Respond only with"f" one word, Product,Activity,Energy use,Transport")
  response=llm.invoke([HumanMessage(content=prompt)])
  category=response.content.strip()
  print(f"Category is {category}")
  state["category"]=category.lower()
  return state

In [ ]:
def input_router(state:dict)->dict:
    cat=state["category"]
    print(cat)
    if("product" in cat ):
        return "Product"
    elif("activity" in cat ):
        return "Activity"
    elif("transport" in cat ):
        return "Transport"
    elif("energy use" in cat):
      return "Energy use"
    else:
        print("Try Again")

In [ ]:
def impact_assessor(state:dict)->dict:
  prompt=(
      f"Give a couple of the environmental impacts of using {state['val']} ")
  response=llm.invoke([HumanMessage(content=prompt)])
  res1=response.content.strip()
  print(f"{res1}")
  #print(type(state))
  state["res1"]=res1
  return state

In [ ]:
def alternative(state:dict)->dict:
  prompt=(
      f"Give a couple of environmental friendly alternatives for {state['val']} ")
  response=llm.invoke([HumanMessage(content=prompt)])
  res=response.content.strip()
  print(f"{res}")
  state["res"]=res
  return state

In [ ]:
builder=StateGraph(dict)
builder.set_entry_point("get_input")
builder.add_node("get_input",get_input)
builder.add_node("input_classifier",input_classifier)
builder.add_node("input_router",input_router)
builder.add_node("impact_assessor",impact_assessor)
builder.add_node("alternative",alternative)

builder.add_edge("get_input","input_classifier")
builder.add_edge("input_classifier","impact_assessor")
#builder.add_edge("input_router","impact_assessor")
builder.add_edge("impact_assessor","alternative")
builder.add_edge("alternative",END)

In [ ]:
graph = builder.compile()
final_state = graph.invoke({})
#print(final_state)

Enter input:coal combustion
Category is Energy use
Coal combustion has significant environmental impacts, including:

1. **Greenhouse gas emissions:**  Burning coal releases large amounts of carbon dioxide (CO2), a major greenhouse gas contributing to climate change.  This is because coal is primarily carbon, and combustion combines it with oxygen from the air to form CO2.  The release of CO2 traps heat in the atmosphere, leading to global warming and its associated effects like sea-level rise, extreme weather events, and disruptions to ecosystems.

2. **Air pollution:** Coal combustion releases various air pollutants, including sulfur dioxide (SO2), nitrogen oxides (NOx), particulate matter (PM), and mercury.  These pollutants contribute to acid rain, respiratory illnesses, and other health problems.  Particulate matter, in particular, can penetrate deep into the lungs and cause serious health issues.  Mercury is a neurotoxin that bioaccumulates in the food chain, posing risks to wild